# ETL MongoDB to BQ
We will create an ETL to transfer mongodb sample data from AirBnB to Bigquery
>Source: https://www.mongodb.com/developer/products/atlas/atlas-sample-datasets/

In [1]:
# import modin.pandas as pd
import pandas as pd
import hashlib
from pathlib import Path
import os
from datetime import datetime
from pymongo import MongoClient
# Set-up google credentials
import time
from google.cloud import bigquery
# Display
pd.set_option("display.max_columns", None)
pd.set_option("expand_frame_repr", False)
# Prefect
from prefect_gcp import GcpCredentials
from prefect import task, flow
from prefect.blocks.system import Secret
from prefect_gcp.cloud_storage import GcsBucket
print('Setup Complete')  

Setup Complete


## Extract Sample AirBnb data from MongoDB

## Function get_mongo-client

In [ ]:
# Define a function to load the MongoClient
# Although the URI string for the Sample Dataset is public
# I will implement this code to obscure the access key.
@flow(log_prints=True, name="get-mongo-client")
def get_mongo_client():
    secret_block = Secret.load("sample-mongodb-uri")
    mongo_uri = secret_block.get()
    mongo_client = MongoClient(mongo_uri)
    return mongo_client

## Sample Datasets
>Extract `sample_airbnb` from mongodb sample_database

# Function extract-collection

In [ ]:
# Define a function to extract all the documents inside the collection
@flow(log_prints=True, name="get-collection-from-mongodb")
def extract_collection(db_name: str, coll_name: str) -> pd.DataFrame:
    mongo_client = get_mongo_client()
    db = mongo_client[db_name]
    db_coll = db.get_collection(coll_name)
    docs = db_coll.find({})
    df = pd.json_normalize(docs, sep="_")
    return df

# df_airbnb = extract_collection(db_name="sample_airbnb", coll_name="listingsAndReviews")
# df_airbnb.head()

In [ ]:
# df_airbnb.loc[~df_airbnb["monthly_price"].isna()]

In [ ]:
# df_airbnb.info(memory_usage="deep")

In [ ]:
# df_airbnb.columns.to_list()

#### df-reviews

In [ ]:
# # Exploring Reviews column
# df_reviews = pd.json_normalize(df_airbnb[["reviews"]].explode(column="reviews").reviews)
# df_reviews.head()

In [ ]:
# df_reviews.info()

In [ ]:
# df_airbnb.head(n=2)

# Function tweak-df

In [ ]:
def tweak_df(df: pd.DataFrame) -> pd.DataFrame:
    # This is a challenge when doing pipeline, we are getting error
    # when uploading to GCS or BQ. Thats why we have to convert it.

    # This will convert Decimal128 to float and object to string
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
        elif pd.api.types.is_numeric_dtype(df[col]) and not pd.api.types.is_bool_dtype(
            df[col]
        ):
            df[col] = pd.to_numeric(df[col], errors="ignore", downcast="float")

    # This will convert float to int
    df["accommodates"] = df["accommodates"].astype(int)
    df["number_of_reviews"] = df["number_of_reviews"].astype(int)
    df["host_host_listings_count"] = df["host_host_listings_count"].astype(int)
    df["host_host_total_listings_count"] = df["host_host_total_listings_count"].astype(
        int
    )
    df["availability_availability_30"] = df["availability_availability_30"].astype(int)
    df["availability_availability_60"] = df["availability_availability_60"].astype(int)
    df["availability_availability_90"] = df["availability_availability_90"].astype(int)
    df["availability_availability_365"] = df["availability_availability_365"].astype(
        int
    )

    # This will convert object/string to float
    df["price"] = df["price"].astype(float)
    df["security_deposit"] = df["security_deposit"].astype(float)
    df["cleaning_fee"] = df["cleaning_fee"].astype(float)
    df["extra_people"] = df["extra_people"].astype(float)
    df["weekly_price"] = df["weekly_price"].astype(float)
    df["monthly_price"] = df["weekly_price"].astype(float)
    df["host_host_response_rate"] = df["host_host_response_rate"].astype(float)

    # Add a unique id and inserted at before uploading to GCS and BQ
    # We can use this as reference we run deduplicate
    df = df.assign(
        _record_hash=list(
            map(lambda x: hashlib.sha1(str(x).encode("utf-8")).hexdigest(), df["_id"])
        ),
        _bq_inserted_at=datetime.now(),
    )

    return df


# df_ = tweak_df(df_airbnb)
# df_.head()

#  amenities=pd.DataFrame(df["amenities"].tolist())


In [ ]:
# df_.info()

# Function write-to-gcs

In [ ]:
# Convert camelCase to split_case
def convert_to_split_case(text: str) -> str:
    mod_string = list(map(lambda x: '_' + x if x.isupper() else x, text))
    join_string = "".join(mod_string).lower().rstrip("_")
    return join_string

# coll_name="listingsAndReviews"
# convert_to_split_case(coll_name)

In [ ]:
@task(log_prints=True, name="write-to-gcs", retries=3)
def write_to_gcs(df: pd.DataFrame, db_name: str, coll_name: str) -> None:
    directory = Path(f"{db_name}")
    converted_coll_name = convert_to_split_case(coll_name)
    path_name = directory / f"{converted_coll_name}.parquet.snappy"
    try:
        # directory.mkdir()
        os.makedirs(directory, exist_ok=True)
        gcs_block = GcsBucket.load("prefect-gcs-block-airbnb")
        gcs_block.upload_from_dataframe(
            df, to_path=path_name, serialization_format="parquet_snappy"
        )
    except OSError as error:
        print(error)

    print("Loaded data to GCS...Hooray!")
    return

# write_to_gcs(df_, db_name="sample_airbnb", coll_name="listingsAndReviews")


# write-gcs-to-bq

## Function write-to-bq (optional)

In [ ]:
# # Define a function that will create the BQ dataset and create a blank Dataframe
# @flow(log_prints=True, name="write-to-bq")
# def write_to_bq(df, db_name: str, coll_name:str) -> None:
#     gcp_credentials_block = GcpCredentials.load("prefect-gcs-2023-creds")
#     converted_coll_name = convert_to_split_case(coll_name)
#     df.to_gbq(
#         destination_table=f"{db_name}.{converted_coll_name}",
#         project_id="dtc-de-2023",
#         credentials=gcp_credentials_block.get_credentials_from_service_account(),
#         if_exists="append",
#         location="asia-southeast1",
#         table_schema=[
#             {"name": "_id", "type": "STRING"},
#             {"name": "listing_url", "type": "STRING"},
#             {"name": "name", "type": "STRING"},
#             {"name": "summary", "type": "STRING"},
#             {"name": "space", "type": "STRING"},
#             {"name": "description", "type": "STRING"},
#             {"name": "neighborhood_overview", "type": "STRING"},
#             {"name": "notes", "type": "STRING"},
#             {"name": "transit", "type": "STRING"},
#             {"name": "access", "type": "STRING"},
#             {"name":  "interaction", "type": "STRING"},
#             {"name": "house_rules", "type": "STRING"},
#             {"name": "property_type", "type": "STRING"},
#             {"name": "room_type", "type": "STRING"},
#             {"name": "bed_type", "type": "STRING"},
#             {"name": "minimum_nights", "type": "INTEGER"},
#             {"name": "maximum_nights", "type": "INTEGER"},
#             {"name": "cancellation_policy", "type": "STRING"},
#             {"name": "last_scraped", "type": "TIMESTAMP"},
#             {"name": "calendar_last_scraped", "type": "TIMESTAMP"},
#             {"name": "first_review", "type": "TIMESTAMP"},
#             {"name": "last_review", "type": "TIMESTAMP"},
#             {"name": "accommodates", "type": "INTEGER"},
#             {"name": "bedrooms", "type": "FLOAT"},
#             {"name": "beds", "type": "FLOAT"},
#             {"name": "number_of_reviews", "type": "INTEGER"},
#             {"name": "bathrooms", "type": "INTEGER"},
#             {"name": "amenities", "type": "STRING"},
#             {"name": "price", "type": "FLOAT"},
#             {"name": "security_deposit", "type": "FLOAT"},
#             {"name": "cleaning_fee", "type": "FLOAT"},
#             {"name": "extra_people", "type": "INTEGER"},
#             {"name": "guests_included", "type": "INTEGER"},
#             {"name": "reviews", "type": "STRING"},
#             {"name": "images_thumbnail_url", "type": "STRING"},
#             {"name": "images_medium_url", "type": "STRING"},
#             {"name": "images_picture_url", "type": "STRING"},
#             {"name": "images_xl_picture_url", "type": "STRING"},
#             {"name": "host_host_id", "type": "STRING"},
#             {"name": "host_host_url", "type": "STRING"},
#             {"name": "host_host_name", "type": "STRING"},
#             {"name": "host_host_location", "type": "STRING"},
#             {"name": "host_host_about", "type": "STRING"},
#             {"name": "host_host_response_time", "type": "STRING"},
#             {"name": "host_host_thumbnail_url", "type": "STRING"},
#             {"name": "host_host_picture_url", "type": "STRING"},
#             {"name": "host_host_neighbourhood", "type": "STRING"},
#             {"name": "host_host_response_rate", "type": "FLOAT"},
#             {"name": "host_host_is_superhost", "type": "BOOLEAN"},
#             {"name": "host_host_has_profile_pic", "type": "BOOLEAN"},
#             {"name": "host_host_identity_verified", "type": "BOOLEAN"},
#             {"name": "host_host_listings_count", "type": "INTEGER"},
#             {"name": "host_host_total_listings_count", "type": "INTEGER"},
#             {"name": "host_host_verifications", "type": "STRING"},
#             {"name": "address_street", "type": "STRING"},
#             {"name": "address_suburb", "type": "STRING"},
#             {"name": "address_government_area", "type": "STRING"},
#             {"name": "address_market", "type": "STRING"},
#             {"name": "address_country", "type": "STRING"},
#             {"name": "address_country_code", "type": "STRING"},
#             {"name": "address_location_type", "type": "STRING"},
#             {"name": "address_location_coordinates", "type": "STRING"},
#             {"name": "address_location_is_location_exact", "type": "BOOLEAN"},
#             {"name": "availability_availability_30", "type": "INTEGER"},
#             {"name": "availability_availability_60", "type": "INTEGER"},
#             {"name": "availability_availability_90", "type": "INTEGER"},
#             {"name": "availability_availability_365", "type": "INTEGER"},
#             {"name": "review_scores_review_scores_accuracy", "type": "FLOAT"},
#             {"name": "review_scores_review_scores_cleanliness", "type": "FLOAT"},
#             {"name": "review_scores_review_scores_checkin", "type": "FLOAT"},
#             {"name": "review_scores_review_scores_communication", "type": "FLOAT"},
#             {"name": "review_scores_review_scores_location", "type": "FLOAT"},
#             {"name": "review_scores_review_scores_value", "type": "FLOAT"},
#             {"name": "review_scores_review_scores_rating", "type": "FLOAT"},
#             {"name": "weekly_price", "type": "FLOAT"},
#             {"name": "monthly_price", "type": "FLOAT"},
#             {"name": "reviews_per_month", "type": "FLOAT"},
#             {"name": "_record_hash", "type": "STRING"},
#             {"name": "_bq_inserted_at", "type": "TIMESTAMP"},
#         ],
#     )
#     return


# write_to_bq(df=df_, db_name="sample_airbnb", coll_name="listingsAndReviews")


## Function create-bq-dataset

In [ ]:
# Define a function that will create the BQ dataset and create a blank Dataframe
@flow(log_prints=True, name="write-mongodb-to-bq")
def create_bq_dataset(db_name: str, coll_name: str) -> None:
    gcp_credentials_block = GcpCredentials.load("prefect-gcs-2023-creds")
    converted_coll_name = convert_to_split_case(coll_name)
    df = pd.DataFrame()
    df.to_gbq(
        destination_table=f"{db_name}.{converted_coll_name}",
        project_id="dtc-de-2023",
        credentials=gcp_credentials_block.get_credentials_from_service_account(),
        if_exists="append",
        location="asia-southeast1",
    )
    return

# create_dataset(db_name="sample_airbnb", coll_name="listingsAndReviews")

## Function get-biqquery-client

In [ ]:
# Get bigquery_client
@task(log_prints=True, name="get-bigquery-client")
def get_bigquery_client():
    gcp_creds_block = GcpCredentials.load("prefect-gcs-2023-creds")
    gcp_creds = gcp_creds_block.get_credentials_from_service_account()
    client = bigquery.Client(credentials=gcp_creds)
    return client

## Function write-gcs-to-bq

In [ ]:
# schema=[
#             bigquery.SchemaField("_id", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("listing_url", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("name", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("summary", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("space", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("neighborhood_overview", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("notes", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("transit", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("access", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("interaction", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("house_rules", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("property_type", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("room_type", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("bed_type", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("minimum_nights", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("maximum_nights", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("cancellation_policy", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("last_scraped", "TIMESTAMP", mode="NULLABLE"),
#             bigquery.SchemaField("calendar_last_scraped", "TIMESTAMP", mode="NULLABLE"),
#             bigquery.SchemaField("first_review", "TIMESTAMP", mode="NULLABLE"),
#             bigquery.SchemaField("last_review", "TIMESTAMP", mode="NULLABLE"),
#             bigquery.SchemaField("accommodates", "INTEGER", mode="NULLABLE"),
#             bigquery.SchemaField("bedrooms", "FLOAT", mode="NULLABLE"),
#             bigquery.SchemaField("beds", "FLOAT", mode="NULLABLE"),
#             bigquery.SchemaField("number_of_reviews", "INTEGER", mode="NULLABLE"),
#             bigquery.SchemaField("bathrooms", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("amenities", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("price", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("security_deposit", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("cleaning_fee", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("extra_people", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("guests_included", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("reviews", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("images_thumbnail_url", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("images_medium_url", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("images_picture_url", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("images_xl_picture_url", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("host_host_id", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("host_host_url", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("host_host_name", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("host_host_location", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("host_host_about", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("host_host_response_time", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("host_host_thumbnail_url", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("host_host_picture_url", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("host_host_neighbourhood", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("host_host_response_rate", "FLOAT", mode="NULLABLE"),
#             bigquery.SchemaField("host_host_is_superhost", "BOOLEAN", mode="NULLABLE"),
#             bigquery.SchemaField(
#                 "host_host_has_profile_pic", "BOOLEAN", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "host_host_identity_verified", "BOOLEAN", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "host_host_listings_count", "INTEGER", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "host_host_total_listings_count", "INTEGER", mode="NULLABLE"
#             ),
#             bigquery.SchemaField("host_host_verifications", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("address_street", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("address_suburb", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("address_government_area", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("address_market", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("address_country", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("address_country_code", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("address_location_type", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField(
#                 "address_location_coordinates", "STRING", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "address_location_is_location_exact", "BOOLEAN", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "availability_availability_30", "INTEGER", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "availability_availability_60", "INTEGER", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "availability_availability_90", "INTEGER", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "availability_availability_365", "INTEGER", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "review_scores_review_scores_accuracy", "FLOAT", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "review_scores_review_scores_cleanliness", "FLOAT", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "review_scores_review_scores_checkin", "FLOAT", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "review_scores_review_scores_communication", "FLOAT", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "review_scores_review_scores_location", "FLOAT", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "review_scores_review_scores_value", "FLOAT", mode="NULLABLE"
#             ),
#             bigquery.SchemaField(
#                 "review_scores_review_scores_rating", "FLOAT", mode="NULLABLE"
#             ),
#             bigquery.SchemaField("weekly_price", "FLOAT", mode="NULLABLE"),
#             bigquery.SchemaField("monthly_price", "FLOAT", mode="NULLABLE"),
#             bigquery.SchemaField("reviews_per_month", "FLOAT", mode="NULLABLE"),
#             bigquery.SchemaField("_record_hash", "STRING", mode="NULLABLE"),
#             bigquery.SchemaField("_bq_inserted_at", "TIMESTAMP", mode="NULLABLE"),
#         ]

In [ ]:
# Upload data from GCS to BigQuery
@flow(log_prints=True, name="write-gcs-to-bq")
def write_gcs_to_bq(db_name: str, coll_name: str) -> None:
    client = get_bigquery_client()
    converted_coll_name = convert_to_split_case(coll_name)
    table_id = f"dtc-de-2023.{db_name}.{converted_coll_name}"

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,
        autodetect=True,
    )
    # Dont forget to run TERRAFORM to create the bucket
    # OR create bucket in https://console.cloud.google.com/storage/create-bucket
    uri = f"gs://airbnb-gcs-bucket/{db_name}/{converted_coll_name}.parquet.snappy"
    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = client.get_table(table_id)
    print(f"Loaded {destination_table.num_rows} rows.")


# write_gcs_to_bq(db_name="sample_airbnb", coll_name="listingsAndReviews")


## Function dedup

In [ ]:
# Remove duplicates
@flow(log_prints=True, name="removing-duplicates-from-bq")
def deduplicate_data():
    client = get_bigquery_client()
    # this will remove the duplicates
    query_dedup = """ 
            -- CREATE A CTE TABLE
            CREATE OR REPLACE TABLE 
                `dtc-de-2023.sample_airbnb.listings_and_reviews` AS 
            WITH 
                CTE1 AS (
                SELECT 
                    *, 
                    ROW_NUMBER() OVER(
                                    PARTITION BY _id 
                                    ORDER BY _bq_inserted_at) AS latest_row
                FROM `dtc-de-2023.sample_airbnb.listings_and_reviews`)

            -- FETCH ONLY THE LATEST ROW WHICH IS THE LATEST BQ INSERTED TIMESTAMP
            SELECT * EXCEPT (latest_row)
            FROM CTE1
            WHERE latest_row = 1 
            """
    # limit query to 10GB or priority=bigquery.QueryPriority.BATCH
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
    
    
    # query
    query_job = client.query(query_dedup, job_config=safe_config)

    # Check progress
    while query_job.state == "RUNNING":
        query_job = client.get_job(query_job.job_id, location=query_job.location)
        time.sleep(1)

        print("Complete removing duplicates")
        print(f"Job {query_job.job_id} is currently in state {query_job.state}")
    return 

# deduplicate_data()

### Notes:
### In the case of the Airbnb dataset, you could partition the data by date or location, and then cluster the data within each partition based on columns such as the type of property, number of bedrooms, or amenities. This would allow you to quickly and easily analyze subsets of the data that are relevant to your specific analysis or query.

In [ ]:
# Create a table with PARTITION AND CLUSTER
# Query
@flow(log_prints=True, name="create-partition-clustered-bq-table")
def create_partition_clustered_bq_table() -> None:
    print("Creating a separate partition and clustered table")
    client = get_bigquery_client()
    airbnb_part_clus = """
                        CREATE OR REPLACE TABLE 
                            `dtc-de-2023.sample_airbnb.listings_and_reviews_part_clust`
                        PARTITION BY
                            DATE(last_scraped)
                        CLUSTER BY
                            property_type AS (
                                SELECT *
                                FROM `dtc-de-2023.sample_airbnb.listings_and_reviews`
                            )
            """
    # Limit results to 1GB=10**10
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
    # Query
    results_part_clus = client.query(airbnb_part_clus, job_config=safe_config).result()
    print("Done creating partitioned and clustered table")
    return results_part_clus


# create_partition_clustered_bq_table()


# Main

In [ ]:
# Main ETL flow to load MongodDB to BigQuery
@flow(log_prints=True, name="etl-mongodb-to-bq")
def etl_mongodb_to_bq(db_name: str, coll_name: str):
    # Extract listings and reviews
    df = extract_collection(db_name, coll_name)
    # Tweak df
    df_ = tweak_df(df)
    # Upload to GCS
    write_to_gcs(df_, db_name, coll_name)
    # Create BQ dataset
    create_bq_dataset(db_name, coll_name)
    # Upload to BigQuery
    write_gcs_to_bq(db_name, coll_name)
    # Remove duplicates by creating or replacing table and
    # using the latest _bq_inserted_at
    deduplicate_data()
    # Create a separate partition and clustered table
    create_partition_clustered_bq_table()

if __name__=="__main__":
    # Parameters
    db_name="sample_airbnb"
    coll_name="listingsAndReviews"
    # Run Main
    etl_mongodb_to_bq(db_name, coll_name)

# Others

In [ ]:
# table_schema = [
# {"name": '_id', "type": str},
# {"name": 'listing_url', "type": str},
# {"name": 'name', "type": str},
# {"name": 'summary', "type": str},
# {"name": 'space', "type": str},
# {"name": 'description', "type": str},
# {"name": 'neighborhood_overview', "type": str},
# {"name": 'notes', "type": str},
# {"name": 'transit', "type": str},
# {"name": 'access', "type": str},
# {"name": 'interaction', "type": str},
# {"name": 'house_rules', "type": str},
# {"name": 'property_type', "type": str},
# {"name": 'room_type', "type": str},
# {"name": 'bed_type', "type": str},
# {"name": 'minimum_nights', "type": int},
# {"name": 'maximum_nights', "type": int},
# {"name": 'cancellation_policy', "type": str},
# {"name": 'last_scraped', "type": "datetime64[ns]"},
# {"name": 'calendar_last_scraped', "type": "datetime64[ns]"},
# {"name": 'first_review', "type": "datetime64[ns]"},
# {"name": 'last_review', "type": "datetime64[ns]"},
# {"name": 'accommodates', "type": int},
# {"name": 'bedrooms', "type": float},
# {"name": 'beds', "type": float},
# {"name": 'number_of_reviews', "type": int},
# {"name": 'bathrooms', "type": int},
# {"name": 'amenities', "type": str},
# {"name": 'price', "type": float},
# {"name": 'security_deposit', "type": float},
# {"name": 'cleaning_fee', "type": float},
# {"name": 'extra_people', "type": int},
# {"name": 'guests_included', "type": int},
# {"name": 'reviews', "type": str},
# {"name": 'images_thumbnail_url', "type": str},
# {"name": 'images_medium_url', "type": str},
# {"name": 'images_picture_url', "type": str},
# {"name": 'images_xl_picture_url', "type": str},
# {"name": 'host_host_id', "type": str},
# {"name": 'host_host_url', "type": str},
# {"name": 'host_host_name', "type": str},
# {"name": 'host_host_location', "type": str},
# {"name": 'host_host_about', "type": str},
# {"name": 'host_host_response_time', "type": str},
# {"name": 'host_host_thumbnail_url', "type": str},
# {"name": 'host_host_picture_url', "type": str},
# {"name": 'host_host_neighbourhood', "type": str},
# {"name": 'host_host_response_rate', "type": float},
# {"name": 'host_host_is_superhost', "type": bool},
# {"name": 'host_host_has_profile_pic', "type": bool},
# {"name": 'host_host_identity_verified', "type": bool},
# {"name": 'host_host_listings_count', "type": int},
# {"name": 'host_host_total_listings_count', "type": int},
# {"name": 'host_host_verifications', "type": str},
# {"name": 'address_street', "type": str},
# {"name": 'address_suburb', "type": str},
# {"name": 'address_government_area', "type": str},
# {"name": 'address_market', "type": str},
# {"name": 'address_country', "type": str},
# {"name": 'address_country_code', "type": str},
# {"name": 'address_location_type', "type": str},
# {"name": 'address_location_coordinates', "type": str},
# {"name": 'address_location_is_location_exact', "type": bool},
# {"name": 'availability_availability_30', "type": int},
# {"name": 'availability_availability_60', "type": int},
# {"name": 'availability_availability_90', "type": int},
# {"name": 'availability_availability_365', "type": int},
# {"name": 'review_scores_review_scores_accuracy', "type": float},
# {"name": 'review_scores_review_scores_cleanliness', "type": float},
# {"name": 'review_scores_review_scores_checkin', "type": float},
# {"name": 'review_scores_review_scores_communication', "type": float},
# {"name": 'review_scores_review_scores_location', "type": float},
# {"name": 'review_scores_review_scores_value', "type": float},
# {"name": 'review_scores_review_scores_rating', "type": float},
# {"name": 'weekly_price', "type": float},
# {"name": 'monthly_price', "type": float},
# {"name": 'reviews_per_month', "type": float},
# {"name": '_record_hash', "type": str},
# {"name": '_bq_inserted_at', "type": "datetime64[ns]"},]

# # print(table_schema)
